In [142]:
# %reset
# 'http://www.bseindia.com/download/BhavCopy/Equity/EQ230110_CSV.ZIP'
import datetime as dt
from datetime import datetime
import pandas as pd
from io import BytesIO
from zipfile import ZipFile
import os
import numpy as np
data_folder="D:/SoftwareWebApps/Python/python_wb/Stocks/Data/NSE/"
dirlist=os.listdir(data_folder) # returns list
from sqlalchemy import create_engine, MetaData # database connection
# DB = {
#     'drivername': 'mysql',
#     'servername': '192.185.128.28', #dynamic server. Will change 
#     'port': '3306',
#     'username': 'thogativ_ab',
#     'password': 'Arun!23',
#     'database': 'thogativ_sthali',
# #     'driver': 'SQL Server Native Client 11.0',
#     'trusted_connection': 'yes',  
#     'legacy_schema_aliasing': False
# }
DB = {
    'drivername': 'mysql',
    'servername': 'localhost', #dynamic server. Will change 
    'port': '3306',
    'username': 'root',
    'password': 'root',
    'database': 'stocks_nse',
#     'driver': 'SQL Server Native Client 11.0',
    'trusted_connection': 'yes',  
    'legacy_schema_aliasing': False
}

db_engine = create_engine(DB['drivername'] + '://' + DB['username'] +':'+ DB['password'] +
                       '@'+ DB['servername'] +':'+ DB['port']+ '/' + DB['database']+'?charset=utf8', encoding="utf-8")  

In [153]:
# dirlist.sort()
DL=pd.DataFrame(dirlist)
# tstamp=pd.Series;
tstam=[]
i=0
for fname in dirlist:
    tstam.append(datetime.strptime(fname[2:11],'%d%b%Y'))
    
DL['tstamp']=tstam   
DL=DL.sort_values(by=['tstamp'])
dirlist=DL[0].tolist()

In [143]:
# drop_database(db_engine.url)
# if(not database_exists(db_engine.url)):
#     print('yes')

In [144]:
from sqlalchemy_utils.functions import create_database, drop_database, database_exists
if(not database_exists(db_engine.url)):
    create_database(db_engine.url)


In [137]:
from sqlalchemy import Column, DateTime, String, Integer, ForeignKey, func,UniqueConstraint
from sqlalchemy.orm import relationship, backref
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.exc import IntegrityError
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.dialects.mysql import \
        BIGINT, BINARY, BIT, BLOB, BOOLEAN, CHAR, DATE, \
        DATETIME, DECIMAL, DECIMAL, DOUBLE, ENUM, FLOAT, INTEGER, \
        LONGBLOB, LONGTEXT, MEDIUMBLOB, MEDIUMINT, MEDIUMTEXT, NCHAR, \
        NUMERIC, NVARCHAR, REAL, SET, SMALLINT, TEXT, TIME, TIMESTAMP, \
        TINYBLOB, TINYINT, TINYTEXT, VARBINARY, VARCHAR, YEAR

session = sessionmaker()
session.configure(bind=db_engine)        
p= session()
Base = declarative_base()
extend_existing=True
def stock_class(table_name):
    class Stock(Base):
        
        __tablename__ = table_name
        TIMESTAMP = Column(DATE, primary_key=True)
        OPEN	=Column(FLOAT,unique=False)
        HIGH	=Column(FLOAT,unique=False)
        LOW	=Column(FLOAT,unique=False)
        CLOSE	=Column(FLOAT,unique=False)
        LAST	=Column(FLOAT,unique=False)
        TOTTRDQTY	=Column(Integer,unique=False)
        TOTTRDVAL	=Column(FLOAT,unique=False)
        TOTALTRADES=Column(Integer,unique=False)
        extend_existing=True

    #     __table_args__ = (UniqueConstraint('ContractDescriptor', 'Trade_Date', name='ContractDescriptor_Trade_Date'),)
        def __init__(self, OPEN,HIGH,LOW,CLOSE,LAST,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES):
            """"""
            self.TIMESTAMP=TIMESTAMP
            self. OPEN= OPEN
            self. HIGH= HIGH
            self. LOW= LOW
            self. CLOSE= CLOSE
            self. LAST= LAST
            self. TOTTRDQTY= TOTTRDQTY
            self. TOTTRDVAL= TOTTRDVAL
            self.TOTALTRADES=TOTALTRADES
            extend_existing=True
    return Stock

#         self. OrderRef= OrderRef
# Base.metadata.create_all(db_engine)

In [138]:
def stock_array(table_name):
    empty_array=[]
    return empty_array

def put_entry(table_arrays,symbols, valarray):
    i=0
    default=[]
    for val in valarray:
        table_arrays.setdefault(symbols[i],default)
#         print(table_arrays[symbols[i]],end=': ')
        table_arrays[symbols[i]].append(val)
#         print(val,end=': ')
#         print(table_arrays[symbols[i]])
        i +=1
    return table_arrays
def armarray_table_fns(symbols):
    return    dict((s,stock_array(s)) for s in symbols)
def update_tables(table_arrays,symbols):
    print("{} new arrays being created...  ".format(len(symbols)))
    table_arrays.update(armarray_table_fns(symbols))
    return table_arrays

def arm_table_fns(symbols):
#     i=0
#     table_fns=[]
#     for s in symbols:
#         table_fns.append({s:stock_class(s)})
#         i +=1
    return    dict((s,stock_class(s)) for s in symbols)
def insert_entry(table_fns,symbols, valarray):
    i=0
    for val in valarray:
    #     print(array(val))
#     OPEN,HIGH,LOW,CLOSE,LASTTOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES
        d = table_fns[symbols[i]](val[0],val[1],val[2],val[3],val[4],val[5],val[6],val[7],val[8])
        i += 1

        try:
            p.add(d)
            p.commit()
        except IntegrityError as e:
            p.rollback()
            print('you have already updated this entry no {}'.format(i))
def create_tables(table_fns,symbols):
    extend_existing=True
    print("{} new tables being created...  ".format(len(symbols)))
#     print(symbols)
    ntime=datetime.now()
    table_fns.update(arm_table_fns(symbols))
    Base.metadata.create_all(db_engine)
    print("It took {} time".format(datetime.now()-ntime))
#     print("Done creation!!!!")
    return table_fns
def dump_in_df(table_fns,symbols, valarray):
    i=0
    for val in valarray:
    #     print(array(val))
#     OPEN,HIGH,LOW,CLOSE,LASTTOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES
        d = table_fns[symbols[i]](val[0],val[1],val[2],val[3],val[4],val[5],val[6],val[7],val[8])
        i += 1

        try:
            p.add(d)
            p.commit()
        except IntegrityError as e:
            p.rollback()
            print('you have already updated this entry no {}'.format(i))

In [139]:
k=0 #one if tests for k value be careful
started=datetime.now()
old=datetime.now()
table_fns={}
for file in dirlist:
    
#     print("File: {}".format(file))
    zip_ref = ZipFile(data_folder+file, 'r')
    # zip_ref.extractall('D:/SoftwareWebApps/Python/python_wb/Stocks/unzip')
    # zip_ref.close()
    # with zipfile.ZipFile('file.zip') as zip:
    #     with zip.open('file.csv') as myZip:
    #         df = pd.read_csv(myZip) 
#     try:        
    with zip_ref as my_zip_file:
            with my_zip_file.open(my_zip_file.namelist()[0]) as myZip:
                df=pd.read_csv(myZip)
                if 'Unnamed: 13' in df.columns:
                    df=df.drop(['Unnamed: 13'], axis=1)
                tstamp=datetime.strptime(df['TIMESTAMP'].iloc[0],'%d-%b-%Y')
                df['TIMESTAMP']=tstamp
                symbols=df.SYMBOL.values
    #             symbols=np.append(symbols,'ARUN')
#                 tables=np.asarray(list(Base.metadata.tables.keys())).astype(object)
                tables=np.asarray(list(table_fns.keys())).astype(object)
        
                tbls_not_in_db=np.setdiff1d(symbols,tables)
            
                if(k==0):
                    table_fns= update_tables(table_fns,symbols)

                if(len(tables)>0 and len(tbls_not_in_db)>0):
                    table_fns= update_tables(table_fns,tbls_not_in_db)


                df=df[df.columns[2:-1]].drop('PREVCLOSE',1)
                if len(df.columns)==8:
                    df['TOTALTRADES']=0
                values=df.values
    #             table_fns=arm_table_fns(symbols)
                table_fns=put_entry(table_fns,symbols, values)


    #                 df.to_sql('nse', db_engine, if_exists='append', index=False )
                k+=1
                print(k,end=', ')
                if (k%100==0):            
                    print("{} of {} files written to DB..".format(k,len(dirlist)))
                    nowtime=datetime.now()
                    print("last 100files took ..")
                    print(nowtime-old)
                    print("From first it took ..",end='')
                    print(nowtime-started)
                    old=nowtime
                    
#     except:
#         print("Zip file not good for file: {}".format(file))

135 new arrays being created...  
1, 5 new arrays being created...  
2, 8 new arrays being created...  
3, 7 new arrays being created...  
4, 2 new arrays being created...  
5, 4 new arrays being created...  
6, 6 new arrays being created...  
7, 5 new arrays being created...  
8, 9, 1 new arrays being created...  
10, 2 new arrays being created...  
11, 2 new arrays being created...  
12, 2 new arrays being created...  
13, 14, 15, 1 new arrays being created...  
16, 2 new arrays being created...  
17, 42 new arrays being created...  
18, 9 new arrays being created...  
19, 10 new arrays being created...  
20, 8 new arrays being created...  
21, 7 new arrays being created...  
22, 2 new arrays being created...  
23, 1 new arrays being created...  
24, 3 new arrays being created...  
25, 26, 2 new arrays being created...  
27, 11 new arrays being created...  
28, 7 new arrays being created...  
29, 13 new arrays being created...  
30, 4 new arrays being created...  
31, 9 new arrays be

301, 302, 303, 304, 305, 306, 1 new arrays being created...  
307, 308, 309, 310, 311, 312, 313, 314, 3 new arrays being created...  
315, 316, 317, 1 new arrays being created...  
318, 319, 320, 1 new arrays being created...  
321, 322, 323, 324, 325, 326, 327, 3 new arrays being created...  
328, 329, 330, 331, 332, 1 new arrays being created...  
333, 334, 335, 336, 337, 338, 339, 340, 341, 1 new arrays being created...  
342, 343, 344, 345, 346, 1 new arrays being created...  
347, 1 new arrays being created...  
348, 1 new arrays being created...  
349, 350, 1 new arrays being created...  
351, 352, 353, 354, 355, 1 new arrays being created...  
356, 1 new arrays being created...  
357, 358, 359, 360, 2 new arrays being created...  
361, 362, 1 new arrays being created...  
363, 364, 1 new arrays being created...  
365, 2 new arrays being created...  
366, 367, 368, 369, 370, 2 new arrays being created...  
371, 1 new arrays being created...  
372, 2 new arrays being created...  


902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 5 new arrays being created...  
915, 1 new arrays being created...  
916, 917, 1 new arrays being created...  
918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 1 new arrays being created...  
939, 940, 1 new arrays being created...  
941, 942, 943, 1 new arrays being created...  
944, 945, 946, 947, 948, 949, 950, 1 new arrays being created...  
951, 952, 953, 1 new arrays being created...  
954, 955, 1 new arrays being created...  
956, 957, 958, 1 new arrays being created...  
959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 1 new arrays being created...  
982, 983, 984, 985, 1 new arrays being created...  
986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1000 of 5751 files written to DB..
last 100files took ..
0:00:04.794018
From first it took ..0:00:42.946655
1001, 2 new 

1590, 1591, 1592, 1593, 2 new arrays being created...  
1594, 1595, 1596, 1597, 1 new arrays being created...  
1598, 1599, 1600, 1600 of 5751 files written to DB..
last 100files took ..
0:00:04.597151
From first it took ..0:01:13.425963
1601, 1602, 1603, 1604, 1605, 1606, 1607, 1608, 1609, 1610, 1611, 1612, 1613, 1614, 1615, 1616, 1617, 1618, 1619, 1620, 1621, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629, 1630, 1631, 1 new arrays being created...  
1632, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 1641, 1 new arrays being created...  
1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651, 1652, 1653, 1654, 1655, 1656, 1657, 1658, 1659, 1660, 1661, 1662, 1663, 1664, 1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673, 1674, 1675, 1676, 1677, 1678, 1679, 1680, 1681, 1682, 1683, 4 new arrays being created...  
1684, 1685, 1686, 1687, 2 new arrays being created...  
1688, 1689, 1690, 1691, 1692, 1693, 1694, 1695, 1696, 1697, 1 new arrays being created...  
1698, 1699, 1700, 17

2302, 2303, 2304, 2305, 2306, 2307, 2308, 2309, 2310, 2311, 2 new arrays being created...  
2312, 2313, 2314, 2315, 1 new arrays being created...  
2316, 2317, 2318, 1 new arrays being created...  
2319, 2 new arrays being created...  
2320, 2321, 1 new arrays being created...  
2322, 2323, 2324, 2325, 2326, 2327, 2328, 2329, 1 new arrays being created...  
2330, 2331, 2 new arrays being created...  
2332, 2333, 1 new arrays being created...  
2334, 2335, 2336, 2337, 2 new arrays being created...  
2338, 1 new arrays being created...  
2339, 1 new arrays being created...  
2340, 2341, 2342, 2343, 2344, 2345, 2346, 2347, 1 new arrays being created...  
2348, 2349, 2350, 2351, 2352, 2353, 2354, 2 new arrays being created...  
2355, 2356, 2357, 2358, 2359, 2 new arrays being created...  
2360, 2 new arrays being created...  
2361, 2362, 2363, 2364, 2365, 2 new arrays being created...  
2366, 2 new arrays being created...  
2367, 2 new arrays being created...  
2368, 2369, 2370, 2371, 1 ne

...

3960, 3961, 1 new arrays being created...  
3962, 3963, 3964, 1 new arrays being created...  
3965, 4 new arrays being created...  
3966, 2 new arrays being created...  
3967, 1 new arrays being created...  
3968, 2 new arrays being created...  
3969, 3970, 1 new arrays being created...  
3971, 1 new arrays being created...  
3972, 1 new arrays being created...  
3973, 1 new arrays being created...  
3974, 1 new arrays being created...  
3975, 1 new arrays being created...  
3976, 3977, 2 new arrays being created...  
3978, 1 new arrays being created...  
3979, 1 new arrays being created...  
3980, 3981, 2 new arrays being created...  
3982, 3983, 3984, 1 new arrays being created...  
3985, 3986, 3987, 1 new arrays being created...  
3988, 3989, 2 new arrays being created...  
3990, 1 new arrays being created...  
3991, 3992, 1 new arrays being created...  
3993, 1 new arrays being created...  
3994, 3995, 3996, 1 new arrays being created...  
3997, 2 new arrays being created...  
3998

4354, 4355, 2 new arrays being created...  
4356, 4357, 4358, 4359, 4360, 1 new arrays being created...  
4361, 4362, 4363, 1 new arrays being created...  
4364, 1 new arrays being created...  
4365, 4366, 4367, 1 new arrays being created...  
4368, 1 new arrays being created...  
4369, 4370, 4371, 4372, 4373, 4374, 1 new arrays being created...  
4375, 4376, 4377, 4378, 4379, 4380, 4381, 4382, 4383, 4384, 4385, 4386, 1 new arrays being created...  
4387, 2 new arrays being created...  
4388, 1 new arrays being created...  
4389, 4390, 1 new arrays being created...  
4391, 4392, 4393, 4394, 1 new arrays being created...  
4395, 4396, 4397, 4398, 4399, 2 new arrays being created...  
4400, 4400 of 5751 files written to DB..
last 100files took ..
0:00:12.339521
From first it took ..0:04:37.167721
4401, 4402, 4403, 1 new arrays being created...  
4404, 4405, 1 new arrays being created...  
4406, 1 new arrays being created...  
4407, 4 new arrays being created...  
4408, 4409, 4410, 4411, 

4867, 4868, 4869, 4870, 4871, 1 new arrays being created...  
4872, 4873, 2 new arrays being created...  
4874, 1 new arrays being created...  
4875, 4876, 1 new arrays being created...  
4877, 4878, 4879, 4880, 4881, 4882, 4883, 4884, 4885, 4886, 4887, 4888, 4889, 4890, 4891, 1 new arrays being created...  
4892, 4893, 4894, 4895, 4896, 4897, 4898, 1 new arrays being created...  
4899, 4900, 4900 of 5751 files written to DB..
last 100files took ..
0:00:17.113954
From first it took ..0:05:44.707998
4901, 2 new arrays being created...  
4902, 4903, 1 new arrays being created...  
4904, 1 new arrays being created...  
4905, 3 new arrays being created...  
4906, 1 new arrays being created...  
4907, 1 new arrays being created...  
4908, 4909, 1 new arrays being created...  
4910, 4911, 2 new arrays being created...  
4912, 1 new arrays being created...  
4913, 4914, 1 new arrays being created...  
4915, 4916, 4917, 4918, 4919, 4920, 4921, 2 new arrays being created...  
4922, 4923, 4924, 

5292, 5293, 2 new arrays being created...  
5294, 1 new arrays being created...  
5295, 2 new arrays being created...  
5296, 1 new arrays being created...  
5297, 3 new arrays being created...  
5298, 5299, 1 new arrays being created...  
5300, 5300 of 5751 files written to DB..
last 100files took ..
0:00:15.614776
From first it took ..0:06:46.811027
5301, 1 new arrays being created...  
5302, 5303, 5304, 5305, 5306, 2 new arrays being created...  
5307, 1 new arrays being created...  
5308, 1 new arrays being created...  
5309, 5310, 5311, 5312, 5313, 5314, 1 new arrays being created...  
5315, 5316, 1 new arrays being created...  
5317, 5318, 1 new arrays being created...  
5319, 3 new arrays being created...  
5320, 3 new arrays being created...  
5321, 1 new arrays being created...  
5322, 2 new arrays being created...  
5323, 5324, 5325, 1 new arrays being created...  
5326, 3 new arrays being created...  
5327, 2 new arrays being created...  
5328, 5329, 1 new arrays being creat

5637, 2 new arrays being created...  
5638, 5639, 2 new arrays being created...  
5640, 1 new arrays being created...  
5641, 2 new arrays being created...  
5642, 5 new arrays being created...  
5643, 1 new arrays being created...  
5644, 2 new arrays being created...  
5645, 2 new arrays being created...  
5646, 2 new arrays being created...  
5647, 3 new arrays being created...  
5648, 1 new arrays being created...  
5649, 2 new arrays being created...  
5650, 1 new arrays being created...  
5651, 1 new arrays being created...  
5652, 5653, 2 new arrays being created...  
5654, 5655, 1 new arrays being created...  
5656, 5657, 5658, 1 new arrays being created...  
5659, 1 new arrays being created...  
5660, 5661, 1 new arrays being created...  
5662, 5663, 1 new arrays being created...  
5664, 5665, 5666, 5667, 1 new arrays being created...  
5668, 3 new arrays being created...  
5669, 4 new arrays being created...  
5670, 5671, 1 new arrays being created...  
5672, 1 new arrays bei

In [145]:

tables=np.asarray(list(table_fns.keys())).astype(object)
table_functions={}
create_tables(table_functions,tables)
stime=datetime.now()
for tbl in tables:
    print(tbl)
    try:
        ntime=datetime.now()
        
        pd.DataFrame(table_fns.get(tbl),
                     columns=['OPEN','HIGH','LOW','CLOSE','LAST','TOTTRDQTY','TOTTRDVAL','TIMESTAMP','TOTALTRADES']).to_sql(tbl.lower(), con=db_engine, if_exists='replace',index=False)
        print("It took {} time".format(datetime.now()-ntime))
    except:
        print("It exists")
print("Total time it took {} ".format(datetime.now()-stime))

3961 new tables being created...  


D:\adiarun\Anaconda3\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Stock, and will be replaced in the string-lookup table.
  item.__name__


It took 0:24:27.779741 time
ABB
It took 0:00:04.346711 time
ACC
It took 0:00:02.777535 time
AHMEDELEC
It took 0:00:03.825990 time
ALFALAVAL
It took 0:00:02.013986 time
APOLLOTYRE
It took 0:00:05.620124 time
APPLEIND
It took 0:00:01.756456 time
ARVINDMILL
It took 0:00:02.133453 time
ASHOKLEY
It took 0:00:03.040140 time
ASIANPAINT
It took 0:00:03.933462 time
BAJAJAUTO
It took 0:00:02.008193 time
BARODARAYN
It took 0:00:02.378004 time
BASF
It took 0:00:03.000378 time
BATAINDIA
It took 0:00:02.719713 time
BHARATFORG
It took 0:00:02.239874 time
BHEL
It took 0:00:03.053692 time
BOMDYEING
It took 0:00:02.874428 time
BONGAIREFN
It took 0:00:02.853216 time
BOOTSPHARM
It took 0:00:02.482354 time
BRITANNIA
It took 0:00:05.820048 time
BSES
It took 0:00:02.716921 time
CARRIERAIR
It took 0:00:01.834686 time
CASTROL
It took 0:00:05.160816 time
CEAT
It took 0:00:02.224678 time
CENTENKA
It took 0:00:04.617530 time
CENTURYTEX
It took 0:00:03.271750 time
CESC
It took 0:00:02.142853 time
CHAMBLFERT
It too

It took 0:00:01.932704 time
KOPRAN
It took 0:00:03.778892 time
NIIT
It took 0:00:02.138028 time
RAJASSPG
It took 0:00:01.921246 time
SPECIALSTL
It took 0:00:02.785603 time
SREERAYALK
It took 0:00:01.417620 time
SRF
It took 0:00:02.399787 time
USHAINDIA
It took 0:00:02.272796 time
WOOLWORTH
It took 0:00:01.166400 time
GIPCL
It took 0:00:04.074951 time
INDIANORG
It took 0:00:03.458288 time
INVESTCORP
It took 0:00:01.319235 time
LOKHSG
It took 0:00:02.404884 time
MODIRUBBER
It took 0:00:03.440151 time
NATHPULP
It took 0:00:02.506905 time
SAMTEL
It took 0:00:03.691484 time
SRFFINANCE
It took 0:00:02.179466 time
TATATIMKEN
It took 0:00:01.526472 time
VARDHMNSPG
It took 0:00:05.671866 time
ADVANIORLI
It took 0:00:02.548753 time
GARDENSILK
It took 0:00:04.759809 time
GLFL
It took 0:00:03.520208 time
GODFRYPHLP
It took 0:00:02.729502 time
MCDOWELL
It took 0:00:02.748487 time
POLYPLEX
It took 0:00:03.224988 time
SANDVIKAS
It took 0:00:02.795408 time
VLSFINANCE
It took 0:00:03.590961 time
HCL-HP

It took 0:00:02.584951 time
NIPPONDENS
It took 0:00:01.399277 time
PRECSNFAST
It took 0:00:02.710455 time
SUDARSCHEM
It took 0:00:04.223271 time
CANFINHOME
It took 0:00:02.457563 time
FAGPRECISN
It took 0:00:02.345577 time
KINETICENG
It took 0:00:02.284646 time
RAJINSTEEL
It took 0:00:01.127223 time
CAPRIHANS
It took 0:00:01.277673 time
COATESIND
It took 0:00:02.595613 time
GOODYEAR
It took 0:00:01.737789 time
KAJARIACER
It took 0:00:04.461721 time
TATATELECM
It took 0:00:02.269884 time
TELEPHNCAB
It took 0:00:01.406155 time
COLORCHEM
It took 0:00:02.329312 time
MAHAVIRSPG
It took 0:00:03.641899 time
MRF
It took 0:00:03.747921 time
NAHARENTP
It took 0:00:01.342057 time
LAXMIMACH
It took 0:00:03.765100 time
NAHARINTL
It took 0:00:01.517955 time
SHREESYNTH
It took 0:00:01.964953 time
SUNDRMFAST
It took 0:00:04.236016 time
BIRLA3M
It took 0:00:03.611641 time
KSBPUMPS
It took 0:00:02.955168 time
MODERNMAL
It took 0:00:02.293754 time
SUPERSPIN
It took 0:00:04.157810 time
CRISIL
It took 0:00

...

It took 0:00:04.612724 time
FTFTF12BGR
It took 0:00:02.378295 time
OBEROIREAL
It took 0:00:01.211597 time
BSTRANSCOM
It took 0:00:03.656635 time
GAL
It took 0:00:03.086030 time
PRESTIGE
It took 0:00:03.048183 time
MMFSL
It took 0:00:03.956957 time
RESPONIND
It took 0:00:02.690542 time
HSP141003G
It took 0:00:01.880110 time
COALINDIA
It took 0:00:01.402617 time
ANDHRACEMT
It took 0:00:02.384168 time
IOLCP
It took 0:00:01.815626 time
TECHNO
It took 0:00:02.742723 time
BSLCP1G
It took 0:00:01.522676 time
AXISGOLD
It took 0:00:03.202157 time
GRAVITA
It took 0:00:02.794917 time
RFHF15RGP9
It took 0:00:02.233277 time
CURATECH
It took 0:00:02.521322 time
JINDWORLD
It took 0:00:01.753478 time
PENIND
It took 0:00:03.229554 time
RFHF14RGP2
It took 0:00:01.268712 time
PARRYSUGAR
It took 0:00:02.804420 time
RPPINFRA
It took 0:00:04.130028 time
HMXII14MGR
It took 0:00:01.187252 time
JAMNAAUTO
It took 0:00:01.512519 time
NET4INDIA
It took 0:00:03.077683 time
OBEROIRLTY
It took 0:00:01.957050 time
SU

It took 0:00:03.993050 time
KOKUYOCMLN
It took 0:00:02.570771 time
FTFTF16DO
It took 0:00:01.465725 time
FTFTF16GR
It took 0:00:01.984187 time
V2RETAIL
It took 0:00:02.612165 time
DBSTOCKBRO
It took 0:00:02.293638 time
JAIHINDPRO
It took 0:00:01.406116 time
COMPUSOFT
It took 0:00:02.809146 time
IRFC
It took 0:00:04.797447 time
NEOCORP
It took 0:00:02.255041 time
FMNL
It took 0:00:01.973063 time
MCX
It took 0:00:01.533960 time
ALSTOMT&D
It took 0:00:01.240098 time
IIFLFMPS1G
It took 0:00:01.418816 time
ORIENTREF
It took 0:00:03.862310 time
MINDACORP
It took 0:00:02.843800 time
HUDCO
It took 0:00:04.325363 time
SCHNEIDER
It took 0:00:02.710643 time
TEXINFRA
It took 0:00:02.863409 time
NANDANEXIM
It took 0:00:01.838927 time
PROVOGE
It took 0:00:02.012879 time
CRMFGETF
It took 0:00:01.497940 time
MGOLD
It took 0:00:01.617796 time
NAGAROIL
It took 0:00:03.525644 time
METKORE
It took 0:00:02.235793 time
MTEDUCARE
It took 0:00:02.344051 time
NBCC
It took 0:00:01.404233 time
ORIENTBELL
It took

It took 0:00:03.560851 time
SPICE
It took 0:00:01.380896 time
GULFOILLUB
It took 0:00:03.297046 time
UFTFGR15PX
It took 0:00:01.750119 time
BSLFTPKUDG
It took 0:00:02.485392 time
IDFC-D1216
It took 0:00:01.224704 time
IDFCD1296
It took 0:00:03.712720 time
IPRU8541
It took 0:00:01.666645 time
BDR
It took 0:00:01.824633 time
ISENSEX
It took 0:00:01.620515 time
J301G
It took 0:00:02.044909 time
SPICEMOBI
It took 0:00:01.828725 time
BSLRGES1RD
It took 0:00:02.200368 time
SNOWMAN
It took 0:00:02.568929 time
SHARDACROP
It took 0:00:01.214061 time
J502G
It took 0:00:01.479303 time
SUPERHOUSE
It took 0:00:02.942412 time
SQSBFSI
It took 0:00:02.614494 time
SHEMAROO
It took 0:00:02.679446 time
IPRU2256
It took 0:00:01.805804 time
PROZONINTU
It took 0:00:01.388177 time
SMCSRVRG
It took 0:00:02.733854 time
AXISCADES
It took 0:00:01.481292 time
MRO
It took 0:00:04.390662 time
CIGNITITEC
It took 0:00:01.921959 time
MOMAI
It took 0:00:01.304674 time
SMCSRVIRG
It took 0:00:02.373197 time
SDF36M6GR
It 

It took 0:00:02.178652 time
KELLTONTEC
It took 0:00:02.270158 time
NECCLTD
It took 0:00:03.226229 time
VIDHIDYE
It took 0:00:02.046561 time
BSLCAP21RG
It took 0:00:01.705709 time
SEQUENT
It took 0:00:02.475954 time
GOKULAGRO
It took 0:00:02.006731 time
SIMBHALS
It took 0:00:02.278975 time
KAMDHENU
It took 0:00:01.767069 time
RDEL
It took 0:00:02.134534 time
KDDL
It took 0:00:01.949622 time
HUSYS
It took 0:00:01.671407 time
HCG
It took 0:00:01.806589 time
HECPROJECT
It took 0:00:01.608428 time
PDMJEPAPER
It took 0:00:02.014626 time
WEALTH
It took 0:00:01.976974 time
BHARATWIRE
It took 0:00:01.871776 time
INEOSSTYRO
It took 0:00:01.873667 time
INFIBEAM
It took 0:00:02.344861 time
CRMFCP6RG
It took 0:00:01.680704 time
EQUITAS
It took 0:00:03.626883 time
LTEBFG
It took 0:00:02.717276 time
PALREDTEC
It took 0:00:03.009456 time
THYROCARE
It took 0:00:01.924109 time
UJJIVAN
It took 0:00:02.247169 time
ARROWGREEN
It took 0:00:02.328887 time
DUCON
It took 0:00:01.941602 time
AXISSCRGP
It took 0

It took 0:00:01.931718 time
DOLLAR
It took 0:00:01.920025 time
WB3RD
It took 0:00:01.972312 time
PANACHE
It took 0:00:01.071118 time
INFOBEAN
It took 0:00:01.340447 time
CKPPRODUCT
It took 0:00:02.546375 time
SCHAND
It took 0:00:02.568511 time
ZOTA
It took 0:00:03.202216 time
BHAGYANGR
It took 0:00:01.400989 time
IRBINVIT
It took 0:00:02.936805 time
HITECHCORP
It took 0:00:01.498010 time
M14RG
It took 0:00:03.920119 time
PSPPROJECT
It took 0:00:01.387818 time
JALAN
It took 0:00:01.694204 time
SGBMAY25
It took 0:00:02.467824 time
VSCL
It took 0:00:01.093350 time
INDIGRID
It took 0:00:01.357116 time
SRIRAM
It took 0:00:01.302149 time
FEDDERELEC
It took 0:00:01.190004 time
STARCEMENT
It took 0:00:01.298721 time
LEEL
It took 0:00:02.992216 time
GLOBE
It took 0:00:02.857975 time
TEJASNET
It took 0:00:01.202469 time
ERIS
It took 0:00:01.508448 time
CDSL
It took 0:00:02.824893 time
GTPL
It took 0:00:01.561755 time
ACCORD
It took 0:00:02.751294 time
AUBANK
It took 0:00:01.756386 time
AXISNIFTY

In [152]:
# Base.metadata.tables.keys()
dirlist[-10:]

tables=np.asarray(list(table_fns.keys())).astype(object)
stime=datetime.now()
for tbl in tables:
    print(tbl)
    try:
        ntime=datetime.now()
        
        pd.DataFrame(table_fns.get(tbl),
                     columns=['OPEN','HIGH','LOW','CLOSE','LAST','TOTTRDQTY','TOTTRDVAL','TIMESTAMP','TOTALTRADES']).to_sql(tbl.lower(), con=db_engine, if_exists='replace',index=False)
        print("It took {} time".format(datetime.now()-ntime))
    except:
        print("It exists")
print("Total time it took {} ".format(datetime.now()-stime))

['cm31OCT2003bhav.csv.zip',
 'cm31OCT2005bhav.csv.zip',
 'cm31OCT2006bhav.csv.zip',
 'cm31OCT2007bhav.csv.zip',
 'cm31OCT2008bhav.csv.zip',
 'cm31OCT2011bhav.csv.zip',
 'cm31OCT2012bhav.csv.zip',
 'cm31OCT2013bhav.csv.zip',
 'cm31OCT2014bhav.csv.zip',
 'cm31OCT2017bhav.csv.zip']

In [165]:
# table_fns[1]
pickle_path='D:\SoftwareWebApps\Python\python_wb\Stocks\Data\stock_pickle\\'
stime=datetime.now()
for tbl in tables[0:20]:
    ntime=datetime.now()
#     stock=   {'symbol':tbl,
#         'columns':['OPEN','HIGH','LOW','CLOSE','LAST','TOTTRDQTY','TOTTRDVAL','TIMESTAMP','TOTALTRADES'],
#          'data':table_fns.get(tbl)}
#     stock=   table_fns.get(tbl)
    # Saving the objects:
    with open(pickle_path+tbl+'.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump([table_fns.get(tbl)], f)
        f.close()

    print("It took {} time".format(datetime.now()-ntime))
print("For all It took {} time".format(datetime.now()-stime))
    

For all It took 0:00:00.697116 time


In [196]:
def save_pkl(file,var):
    with open(file, 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(var, f)
        f.close()
def load_pkl(file):
    with open(file, 'rb') as f:  # Python 3: open(..., 'wb')
        var = pickle.load(f)
        f.close()
    return var


['BIGINT',
 'BINARY',
 'BIT',
 'BLOB',
 'BOOLEAN',
 'Base',
 'BytesIO',
 'CHAR',
 'Column',
 'DATE',
 'DATETIME',
 'DB',
 'DECIMAL',
 'DL',
 'DOUBLE',
 'DateTime',
 'ENUM',
 'FLOAT',
 'ForeignKey',
 'INTEGER',
 'In',
 'Integer',
 'IntegrityError',
 'LONGBLOB',
 'LONGTEXT',
 'MEDIUMBLOB',
 'MEDIUMINT',
 'MEDIUMTEXT',
 'MetaData',
 'NCHAR',
 'NUMERIC',
 'NVARCHAR',
 'Out',
 'REAL',
 'SET',
 'SMALLINT',
 'String',
 'TEXT',
 'TIME',
 'TIMESTAMP',
 'TINYBLOB',
 'TINYINT',
 'TINYTEXT',
 'UniqueConstraint',
 'VARBINARY',
 'VARCHAR',
 'YEAR',
 'ZipFile',
 '__builtin__',
 '__builtins__',
 '__name__',
 '_dh',
 '_i',
 '_i134',
 '_i135',
 '_i136',
 '_i137',
 '_i138',
 '_i139',
 '_i140',
 '_i141',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 '_sh',
 'arm_table_fns',
 'armarray_table_fns',
 'backref',
 'create_database',
 'create_engine',
 'create_tables',
 'data_folder',
 'database_exists',
 'datetime',
 'db_engine',
 'declarative_base',
 'df',
 'dirlist',
 'drop_database',
 'dt',
 'dump_in_df',
 'exit',
 'ex

In [203]:
columns=['OPEN','HIGH','LOW','CLOSE','LAST','TOTTRDQTY','TOTTRDVAL','TIMESTAMP','TOTALTRADES']
pickle_path='D:\SoftwareWebApps\Python\python_wb\Stocks\Data\stock_pickle\\'
stime=datetime.now()
save_pkl(pickle_path+'columns_ref.pkl',columns)
save_pkl(pickle_path+'wrtitten_files.pkl',dirlist)
save_pkl(pickle_path+'symbols.pkl',tables)
for tbl in tables:
    # Saving the objects:
    save_pkl(pickle_path+tbl+'.pkl',table_fns.get(tbl))
    i +=1
    print(i,end=', ')
#     print("It took {} time".format(datetime.now()-ntime))
print("For all It took {} time".format(datetime.now()-stime))



4022, 4023, 4024, 4025, 4026, 4027, 4028, 4029, 4030, 4031, 4032, 4033, 4034, 4035, 4036, 4037, 4038, 4039, 4040, 4041, 4042, 4043, 4044, 4045, 4046, 4047, 4048, 4049, 4050, 4051, 4052, 4053, 4054, 4055, 4056, 4057, 4058, 4059, 4060, 4061, 4062, 4063, 4064, 4065, 4066, 4067, 4068, 4069, 4070, 4071, 4072, 4073, 4074, 4075, 4076, 4077, 4078, 4079, 4080, 4081, 4082, 4083, 4084, 4085, 4086, 4087, 4088, 4089, 4090, 4091, 4092, 4093, 4094, 4095, 4096, 4097, 4098, 4099, 4100, 4101, 4102, 4103, 4104, 4105, 4106, 4107, 4108, 4109, 4110, 4111, 4112, 4113, 4114, 4115, 4116, 4117, 4118, 4119, 4120, 4121, 4122, 4123, 4124, 4125, 4126, 4127, 4128, 4129, 4130, 4131, 4132, 4133, 4134, 4135, 4136, 4137, 4138, 4139, 4140, 4141, 4142, 4143, 4144, 4145, 4146, 4147, 4148, 4149, 4150, 4151, 4152, 4153, 4154, 4155, 4156, 4157, 4158, 4159, 4160, 4161, 4162, 4163, 4164, 4165, 4166, 4167, 4168, 4169, 4170, 4171, 4172, 4173, 4174, 4175, 4176, 4177, 4178, 4179, 4180, 4181, 4182, 4183, 4184, 4185, 4186, 4187, 4188

5398, 5399, 5400, 5401, 5402, 5403, 5404, 5405, 5406, 5407, 5408, 5409, 5410, 5411, 5412, 5413, 5414, 5415, 5416, 5417, 5418, 5419, 5420, 5421, 5422, 5423, 5424, 5425, 5426, 5427, 5428, 5429, 5430, 5431, 5432, 5433, 5434, 5435, 5436, 5437, 5438, 5439, 5440, 5441, 5442, 5443, 5444, 5445, 5446, 5447, 5448, 5449, 5450, 5451, 5452, 5453, 5454, 5455, 5456, 5457, 5458, 5459, 5460, 5461, 5462, 5463, 5464, 5465, 5466, 5467, 5468, 5469, 5470, 5471, 5472, 5473, 5474, 5475, 5476, 5477, 5478, 5479, 5480, 5481, 5482, 5483, 5484, 5485, 5486, 5487, 5488, 5489, 5490, 5491, 5492, 5493, 5494, 5495, 5496, 5497, 5498, 5499, 5500, 5501, 5502, 5503, 5504, 5505, 5506, 5507, 5508, 5509, 5510, 5511, 5512, 5513, 5514, 5515, 5516, 5517, 5518, 5519, 5520, 5521, 5522, 5523, 5524, 5525, 5526, 5527, 5528, 5529, 5530, 5531, 5532, 5533, 5534, 5535, 5536, 5537, 5538, 5539, 5540, 5541, 5542, 5543, 5544, 5545, 5546, 5547, 5548, 5549, 5550, 5551, 5552, 5553, 5554, 5555, 5556, 5557, 5558, 5559, 5560, 5561, 5562, 5563, 5564

6768, 6769, 6770, 6771, 6772, 6773, 6774, 6775, 6776, 6777, 6778, 6779, 6780, 6781, 6782, 6783, 6784, 6785, 6786, 6787, 6788, 6789, 6790, 6791, 6792, 6793, 6794, 6795, 6796, 6797, 6798, 6799, 6800, 6801, 6802, 6803, 6804, 6805, 6806, 6807, 6808, 6809, 6810, 6811, 6812, 6813, 6814, 6815, 6816, 6817, 6818, 6819, 6820, 6821, 6822, 6823, 6824, 6825, 6826, 6827, 6828, 6829, 6830, 6831, 6832, 6833, 6834, 6835, 6836, 6837, 6838, 6839, 6840, 6841, 6842, 6843, 6844, 6845, 6846, 6847, 6848, 6849, 6850, 6851, 6852, 6853, 6854, 6855, 6856, 6857, 6858, 6859, 6860, 6861, 6862, 6863, 6864, 6865, 6866, 6867, 6868, 6869, 6870, 6871, 6872, 6873, 6874, 6875, 6876, 6877, 6878, 6879, 6880, 6881, 6882, 6883, 6884, 6885, 6886, 6887, 6888, 6889, 6890, 6891, 6892, 6893, 6894, 6895, 6896, 6897, 6898, 6899, 6900, 6901, 6902, 6903, 6904, 6905, 6906, 6907, 6908, 6909, 6910, 6911, 6912, 6913, 6914, 6915, 6916, 6917, 6918, 6919, 6920, 6921, 6922, 6923, 6924, 6925, 6926, 6927, 6928, 6929, 6930, 6931, 6932, 6933, 6934

In [201]:
columns=load_pkl(pickle_path+'columns_ref.pkl')
dirlist_written=load_pkl(pickle_path+'wrtitten_files.pkl')
symbols=load_pkl(pickle_path+'symbols.pkl')
stime=datetime.now()
stocks={}
for symb in symbols[0:20]:
    ntime=datetime.now()
    stocks.update({symb: load_pkl(pickle_path+tbl+'.pkl')})
#     print("It took {} time".format(datetime.now()-ntime))
print("For all It took {} time".format(datetime.now()-stime))


In [202]:
# for symb in symbols[0:20]:
#     print(symb)
# for tbl in tables[0:20]:
#     print(tbl)
stocks

{'ABB': [array([205.0, 220.0, 205.0, 215.0, 215.0, 500, 106150.0,
         Timestamp('1994-11-03 00:00:00'), 0], dtype=object),
  array([215.0, 215.0, 212.75, 215.0, 215.0, 400, 85812.5,
         Timestamp('1994-11-07 00:00:00'), 0], dtype=object),
  array([217.0, 217.0, 215.0, 215.0, 215.0, 200, 43100.0,
         Timestamp('1994-11-08 00:00:00'), 0], dtype=object),
  array([216.0, 216.0, 208.0, 208.0, 208.0, 150, 31800.0,
         Timestamp('1994-11-09 00:00:00'), 0], dtype=object),
  array([205.0, 206.0, 205.0, 206.0, 206.0, 600, 123400.0,
         Timestamp('1994-11-10 00:00:00'), 0], dtype=object),
  array([205.5, 210.0, 205.0, 208.75, 210.0, 600, 123775.0,
         Timestamp('1994-11-11 00:00:00'), 0], dtype=object),
  array([210.0, 210.0, 208.0, 210.0, 210.0, 750, 156500.0,
         Timestamp('1994-11-14 00:00:00'), 0], dtype=object),
  array([204.0, 208.5, 203.75, 208.5, 208.5, 300, 61687.5,
         Timestamp('1994-11-15 00:00:00'), 0], dtype=object),
  array([214.0, 214.0, 214

In [194]:
symbols,tables

([array(['ABB', 'ACC', 'AHMEDELEC', ..., 'HINDCON', 'EMOFSR1RDP',
         'H3360D30RQ'], dtype=object)],
 array(['ABB', 'ACC', 'AHMEDELEC', ..., 'HINDCON', 'EMOFSR1RDP',
        'H3360D30RQ'], dtype=object))

In [150]:
import pickle

# obj0, obj1, obj2 are created here...
ntime=datetime.now()
# Saving the objects:
with open('table_fns.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([table_fns], f)

print("It took {} time".format(datetime.now()-ntime))




It took 0:01:40.111550 time


In [248]:
for file in dirlist[2:]:
    print(file)
    zip_ref = ZipFile(data_folder+file, 'r')

cm08NOV1994bhav.csv.zip
cm09NOV1994bhav.csv.zip
cm10NOV1994bhav.csv.zip
cm11NOV1994bhav.csv.zip
cm14NOV1994bhav.csv.zip
cm15NOV1994bhav.csv.zip
cm16NOV1994bhav.csv.zip
cm17NOV1994bhav.csv.zip
cm21NOV1994bhav.csv.zip
cm22NOV1994bhav.csv.zip
cm23NOV1994bhav.csv.zip
cm24NOV1994bhav.csv.zip
cm25NOV1994bhav.csv.zip
cm28NOV1994bhav.csv.zip
cm29NOV1994bhav.csv.zip
cm30NOV1994bhav.csv.zip
cm01DEC1994bhav.csv.zip
cm02DEC1994bhav.csv.zip
cm05DEC1994bhav.csv.zip
cm06DEC1994bhav.csv.zip
cm07DEC1994bhav.csv.zip
cm08DEC1994bhav.csv.zip
cm09DEC1994bhav.csv.zip
cm12DEC1994bhav.csv.zip
cm13DEC1994bhav.csv.zip
cm14DEC1994bhav.csv.zip
cm15DEC1994bhav.csv.zip
cm16DEC1994bhav.csv.zip
cm19DEC1994bhav.csv.zip
cm20DEC1994bhav.csv.zip
cm21DEC1994bhav.csv.zip
cm22DEC1994bhav.csv.zip
cm23DEC1994bhav.csv.zip
cm26DEC1994bhav.csv.zip
cm27DEC1994bhav.csv.zip
cm28DEC1994bhav.csv.zip
cm29DEC1994bhav.csv.zip
cm30DEC1994bhav.csv.zip
cm02JAN1995bhav.csv.zip
cm03JAN1995bhav.csv.zip
cm04JAN1995bhav.csv.zip
cm05JAN1995bhav.

cm03APR1996bhav.csv.zip
cm04APR1996bhav.csv.zip
cm08APR1996bhav.csv.zip
cm09APR1996bhav.csv.zip
cm10APR1996bhav.csv.zip
cm11APR1996bhav.csv.zip
cm12APR1996bhav.csv.zip
cm15APR1996bhav.csv.zip
cm16APR1996bhav.csv.zip
cm17APR1996bhav.csv.zip
cm18APR1996bhav.csv.zip
cm19APR1996bhav.csv.zip
cm22APR1996bhav.csv.zip
cm23APR1996bhav.csv.zip
cm24APR1996bhav.csv.zip
cm25APR1996bhav.csv.zip
cm26APR1996bhav.csv.zip
cm29APR1996bhav.csv.zip
cm30APR1996bhav.csv.zip
cm01MAY1996bhav.csv.zip
cm02MAY1996bhav.csv.zip
cm03MAY1996bhav.csv.zip
cm06MAY1996bhav.csv.zip
cm07MAY1996bhav.csv.zip
cm08MAY1996bhav.csv.zip
cm09MAY1996bhav.csv.zip
cm10MAY1996bhav.csv.zip
cm13MAY1996bhav.csv.zip
cm14MAY1996bhav.csv.zip
cm15MAY1996bhav.csv.zip
cm16MAY1996bhav.csv.zip
cm17MAY1996bhav.csv.zip
cm20MAY1996bhav.csv.zip
cm21MAY1996bhav.csv.zip
cm22MAY1996bhav.csv.zip
cm23MAY1996bhav.csv.zip
cm24MAY1996bhav.csv.zip
cm27MAY1996bhav.csv.zip
cm28MAY1996bhav.csv.zip
cm29MAY1996bhav.csv.zip
cm30MAY1996bhav.csv.zip
cm31MAY1996bhav.

cm10SEP1997bhav.csv.zip
cm11SEP1997bhav.csv.zip
cm12SEP1997bhav.csv.zip
cm15SEP1997bhav.csv.zip
cm16SEP1997bhav.csv.zip
cm17SEP1997bhav.csv.zip
cm18SEP1997bhav.csv.zip
cm19SEP1997bhav.csv.zip
cm22SEP1997bhav.csv.zip
cm23SEP1997bhav.csv.zip
cm24SEP1997bhav.csv.zip
cm25SEP1997bhav.csv.zip
cm26SEP1997bhav.csv.zip
cm29SEP1997bhav.csv.zip
cm30SEP1997bhav.csv.zip
cm01OCT1997bhav.csv.zip
cm09OCT1997bhav.csv.zip
cm10OCT1997bhav.csv.zip
cm13OCT1997bhav.csv.zip
cm14OCT1997bhav.csv.zip
cm15OCT1997bhav.csv.zip
cm16OCT1997bhav.csv.zip
cm17OCT1997bhav.csv.zip
cm20OCT1997bhav.csv.zip
cm21OCT1997bhav.csv.zip
cm22OCT1997bhav.csv.zip
cm23OCT1997bhav.csv.zip
cm24OCT1997bhav.csv.zip
cm27OCT1997bhav.csv.zip
cm28OCT1997bhav.csv.zip
cm29OCT1997bhav.csv.zip
cm30OCT1997bhav.csv.zip
cm03NOV1997bhav.csv.zip
cm04NOV1997bhav.csv.zip
cm05NOV1997bhav.csv.zip
cm06NOV1997bhav.csv.zip
cm07NOV1997bhav.csv.zip
cm10NOV1997bhav.csv.zip
cm11NOV1997bhav.csv.zip
cm12NOV1997bhav.csv.zip
cm13NOV1997bhav.csv.zip
cm17NOV1997bhav.

cm24FEB1999bhav.csv.zip
cm25FEB1999bhav.csv.zip
cm26FEB1999bhav.csv.zip
cm27FEB1999bhav.csv.zip
cm01MAR1999bhav.csv.zip
cm03MAR1999bhav.csv.zip
cm04MAR1999bhav.csv.zip
cm05MAR1999bhav.csv.zip
cm08MAR1999bhav.csv.zip
cm09MAR1999bhav.csv.zip
cm10MAR1999bhav.csv.zip
cm11MAR1999bhav.csv.zip
cm12MAR1999bhav.csv.zip
cm15MAR1999bhav.csv.zip
cm16MAR1999bhav.csv.zip
cm17MAR1999bhav.csv.zip
cm18MAR1999bhav.csv.zip
cm19MAR1999bhav.csv.zip
cm20MAR1999bhav.csv.zip
cm22MAR1999bhav.csv.zip
cm23MAR1999bhav.csv.zip
cm24MAR1999bhav.csv.zip
cm25MAR1999bhav.csv.zip
cm26MAR1999bhav.csv.zip
cm30MAR1999bhav.csv.zip
cm31MAR1999bhav.csv.zip
cm01APR1999bhav.csv.zip
cm05APR1999bhav.csv.zip
cm06APR1999bhav.csv.zip
cm07APR1999bhav.csv.zip
cm08APR1999bhav.csv.zip
cm09APR1999bhav.csv.zip
cm12APR1999bhav.csv.zip
cm13APR1999bhav.csv.zip
cm15APR1999bhav.csv.zip
cm16APR1999bhav.csv.zip
cm17APR1999bhav.csv.zip
cm19APR1999bhav.csv.zip
cm20APR1999bhav.csv.zip
cm21APR1999bhav.csv.zip
cm22APR1999bhav.csv.zip
cm23APR1999bhav.

cm21AUG2000bhav.csv.zip
cm22AUG2000bhav.csv.zip
cm23AUG2000bhav.csv.zip
cm24AUG2000bhav.csv.zip
cm25AUG2000bhav.csv.zip
cm28AUG2000bhav.csv.zip
cm29AUG2000bhav.csv.zip
cm30AUG2000bhav.csv.zip
cm31AUG2000bhav.csv.zip
cm04SEP2000bhav.csv.zip
cm05SEP2000bhav.csv.zip
cm06SEP2000bhav.csv.zip
cm07SEP2000bhav.csv.zip
cm08SEP2000bhav.csv.zip
cm11SEP2000bhav.csv.zip
cm12SEP2000bhav.csv.zip
cm13SEP2000bhav.csv.zip
cm14SEP2000bhav.csv.zip
cm15SEP2000bhav.csv.zip
cm18SEP2000bhav.csv.zip
cm19SEP2000bhav.csv.zip
cm20SEP2000bhav.csv.zip
cm21SEP2000bhav.csv.zip
cm22SEP2000bhav.csv.zip
cm25SEP2000bhav.csv.zip
cm26SEP2000bhav.csv.zip
cm27SEP2000bhav.csv.zip
cm28SEP2000bhav.csv.zip
cm29SEP2000bhav.csv.zip
cm03OCT2000bhav.csv.zip
cm04OCT2000bhav.csv.zip
cm05OCT2000bhav.csv.zip
cm06OCT2000bhav.csv.zip
cm09OCT2000bhav.csv.zip
cm10OCT2000bhav.csv.zip
cm11OCT2000bhav.csv.zip
cm12OCT2000bhav.csv.zip
cm13OCT2000bhav.csv.zip
cm16OCT2000bhav.csv.zip
cm17OCT2000bhav.csv.zip
cm18OCT2000bhav.csv.zip
cm19OCT2000bhav.

...

cm24FEB2012bhav.csv.zip
cm27FEB2012bhav.csv.zip
cm28FEB2012bhav.csv.zip
cm29FEB2012bhav.csv.zip
cm01MAR2012bhav.csv.zip
cm02MAR2012bhav.csv.zip
cm03MAR2012bhav.csv.zip
cm05MAR2012bhav.csv.zip
cm06MAR2012bhav.csv.zip
cm07MAR2012bhav.csv.zip
cm09MAR2012bhav.csv.zip
cm12MAR2012bhav.csv.zip
cm13MAR2012bhav.csv.zip
cm14MAR2012bhav.csv.zip
cm15MAR2012bhav.csv.zip
cm16MAR2012bhav.csv.zip
cm19MAR2012bhav.csv.zip
cm20MAR2012bhav.csv.zip
cm21MAR2012bhav.csv.zip
cm22MAR2012bhav.csv.zip
cm23MAR2012bhav.csv.zip
cm26MAR2012bhav.csv.zip
cm27MAR2012bhav.csv.zip
cm28MAR2012bhav.csv.zip
cm29MAR2012bhav.csv.zip
cm30MAR2012bhav.csv.zip
cm02APR2012bhav.csv.zip
cm03APR2012bhav.csv.zip
cm04APR2012bhav.csv.zip
cm09APR2012bhav.csv.zip
cm10APR2012bhav.csv.zip
cm11APR2012bhav.csv.zip
cm12APR2012bhav.csv.zip
cm13APR2012bhav.csv.zip
cm16APR2012bhav.csv.zip
cm17APR2012bhav.csv.zip
cm18APR2012bhav.csv.zip
cm19APR2012bhav.csv.zip
cm20APR2012bhav.csv.zip
cm23APR2012bhav.csv.zip
cm24APR2012bhav.csv.zip
cm25APR2012bhav.

cm29JUL2013bhav.csv.zip
cm30JUL2013bhav.csv.zip
cm31JUL2013bhav.csv.zip
cm01AUG2013bhav.csv.zip
cm02AUG2013bhav.csv.zip
cm05AUG2013bhav.csv.zip
cm06AUG2013bhav.csv.zip
cm07AUG2013bhav.csv.zip
cm08AUG2013bhav.csv.zip
cm12AUG2013bhav.csv.zip
cm13AUG2013bhav.csv.zip
cm14AUG2013bhav.csv.zip
cm16AUG2013bhav.csv.zip
cm19AUG2013bhav.csv.zip
cm20AUG2013bhav.csv.zip
cm21AUG2013bhav.csv.zip
cm22AUG2013bhav.csv.zip
cm23AUG2013bhav.csv.zip
cm26AUG2013bhav.csv.zip
cm27AUG2013bhav.csv.zip
cm28AUG2013bhav.csv.zip
cm29AUG2013bhav.csv.zip
cm30AUG2013bhav.csv.zip
cm02SEP2013bhav.csv.zip
cm03SEP2013bhav.csv.zip
cm04SEP2013bhav.csv.zip
cm05SEP2013bhav.csv.zip
cm06SEP2013bhav.csv.zip
cm10SEP2013bhav.csv.zip
cm11SEP2013bhav.csv.zip
cm12SEP2013bhav.csv.zip
cm13SEP2013bhav.csv.zip
cm16SEP2013bhav.csv.zip
cm17SEP2013bhav.csv.zip
cm18SEP2013bhav.csv.zip
cm19SEP2013bhav.csv.zip
cm20SEP2013bhav.csv.zip
cm23SEP2013bhav.csv.zip
cm24SEP2013bhav.csv.zip
cm25SEP2013bhav.csv.zip
cm26SEP2013bhav.csv.zip
cm27SEP2013bhav.

cm02FEB2015bhav.csv.zip
cm03FEB2015bhav.csv.zip
cm04FEB2015bhav.csv.zip
cm05FEB2015bhav.csv.zip
cm06FEB2015bhav.csv.zip
cm09FEB2015bhav.csv.zip
cm10FEB2015bhav.csv.zip
cm11FEB2015bhav.csv.zip
cm12FEB2015bhav.csv.zip
cm13FEB2015bhav.csv.zip
cm16FEB2015bhav.csv.zip
cm18FEB2015bhav.csv.zip
cm19FEB2015bhav.csv.zip
cm20FEB2015bhav.csv.zip
cm23FEB2015bhav.csv.zip
cm24FEB2015bhav.csv.zip
cm25FEB2015bhav.csv.zip
cm26FEB2015bhav.csv.zip
cm27FEB2015bhav.csv.zip
cm28FEB2015bhav.csv.zip
cm02MAR2015bhav.csv.zip
cm03MAR2015bhav.csv.zip
cm04MAR2015bhav.csv.zip
cm05MAR2015bhav.csv.zip
cm09MAR2015bhav.csv.zip
cm10MAR2015bhav.csv.zip
cm11MAR2015bhav.csv.zip
cm12MAR2015bhav.csv.zip
cm13MAR2015bhav.csv.zip
cm16MAR2015bhav.csv.zip
cm17MAR2015bhav.csv.zip
cm18MAR2015bhav.csv.zip
cm19MAR2015bhav.csv.zip
cm20MAR2015bhav.csv.zip
cm23MAR2015bhav.csv.zip
cm24MAR2015bhav.csv.zip
cm25MAR2015bhav.csv.zip
cm26MAR2015bhav.csv.zip
cm27MAR2015bhav.csv.zip
cm30MAR2015bhav.csv.zip
cm31MAR2015bhav.csv.zip
cm01APR2015bhav.

cm01JUL2016bhav.csv.zip
cm04JUL2016bhav.csv.zip
cm05JUL2016bhav.csv.zip
cm07JUL2016bhav.csv.zip
cm08JUL2016bhav.csv.zip
cm11JUL2016bhav.csv.zip
cm12JUL2016bhav.csv.zip
cm13JUL2016bhav.csv.zip
cm14JUL2016bhav.csv.zip
cm15JUL2016bhav.csv.zip
cm18JUL2016bhav.csv.zip
cm19JUL2016bhav.csv.zip
cm20JUL2016bhav.csv.zip
cm21JUL2016bhav.csv.zip
cm22JUL2016bhav.csv.zip
cm25JUL2016bhav.csv.zip
cm26JUL2016bhav.csv.zip
cm27JUL2016bhav.csv.zip
cm28JUL2016bhav.csv.zip
cm29JUL2016bhav.csv.zip
cm01AUG2016bhav.csv.zip
cm02AUG2016bhav.csv.zip
cm03AUG2016bhav.csv.zip
cm04AUG2016bhav.csv.zip
cm05AUG2016bhav.csv.zip
cm08AUG2016bhav.csv.zip
cm09AUG2016bhav.csv.zip
cm10AUG2016bhav.csv.zip
cm11AUG2016bhav.csv.zip
cm12AUG2016bhav.csv.zip
cm16AUG2016bhav.csv.zip
cm17AUG2016bhav.csv.zip
cm18AUG2016bhav.csv.zip
cm19AUG2016bhav.csv.zip
cm22AUG2016bhav.csv.zip
cm23AUG2016bhav.csv.zip
cm24AUG2016bhav.csv.zip
cm25AUG2016bhav.csv.zip
cm26AUG2016bhav.csv.zip
cm29AUG2016bhav.csv.zip
cm30AUG2016bhav.csv.zip
cm31AUG2016bhav.

cm07DEC2017bhav.csv.zip
cm08DEC2017bhav.csv.zip
cm11DEC2017bhav.csv.zip
cm12DEC2017bhav.csv.zip
cm13DEC2017bhav.csv.zip
cm14DEC2017bhav.csv.zip
cm15DEC2017bhav.csv.zip
cm18DEC2017bhav.csv.zip
cm19DEC2017bhav.csv.zip
cm20DEC2017bhav.csv.zip
cm21DEC2017bhav.csv.zip
cm22DEC2017bhav.csv.zip
cm26DEC2017bhav.csv.zip
cm27DEC2017bhav.csv.zip
cm28DEC2017bhav.csv.zip
cm29DEC2017bhav.csv.zip
cm01JAN2018bhav.csv.zip
cm02JAN2018bhav.csv.zip
cm03JAN2018bhav.csv.zip
cm04JAN2018bhav.csv.zip
cm05JAN2018bhav.csv.zip
cm08JAN2018bhav.csv.zip
cm09JAN2018bhav.csv.zip
cm10JAN2018bhav.csv.zip
cm11JAN2018bhav.csv.zip
cm12JAN2018bhav.csv.zip
cm15JAN2018bhav.csv.zip
cm16JAN2018bhav.csv.zip
cm17JAN2018bhav.csv.zip
cm18JAN2018bhav.csv.zip
cm19JAN2018bhav.csv.zip
cm22JAN2018bhav.csv.zip
cm23JAN2018bhav.csv.zip
cm24JAN2018bhav.csv.zip
cm25JAN2018bhav.csv.zip
cm29JAN2018bhav.csv.zip
cm30JAN2018bhav.csv.zip
cm31JAN2018bhav.csv.zip
cm01FEB2018bhav.csv.zip
cm02FEB2018bhav.csv.zip
cm05FEB2018bhav.csv.zip
cm06FEB2018bhav.

In [218]:
mydict={}
mydict.update({'1':'hello','2':'jello'})
gdict={'3':'hello1','a':'jello2'}
mydict.update({'3':'hello1','a':'jello2'})

In [228]:
len(tables)

0

In [211]:
new_tfns[]

{'BROOKBOND': __main__.stock_class.<locals>.Stock,
 'JKCORP': __main__.stock_class.<locals>.Stock,
 'SCI': __main__.stock_class.<locals>.Stock,
 'TATAHYDRO': __main__.stock_class.<locals>.Stock,
 'VOLTAS': __main__.stock_class.<locals>.Stock}

In [202]:
tbls_not_in_db

array(['BROOKBOND', 'JKCORP', 'SCI', 'TATAHYDRO', 'VOLTAS'], dtype=object)

In [212]:
new_tfns[tbls_not_in_db[0]]

__main__.stock_class.<locals>.Stock

In [37]:
genomes = ["A", "B"]
tbl=[]
# tables = {x: build_genome_table(x) for x in genomes}
# mytable1=get_class('table1')
# mytable2=get_class('table2')
i=0
for s in genomes:
    tbl.append(stock_class(s))
    i +=1
Base.metadata.create_all(db_engine)

D:\adiarun\Anaconda3\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Stock, and will be replaced in the string-lookup table.
  item.__name__


In [70]:
# df.SYMBOL.values


In [71]:
values.shape[1]
# len(values)
values[1]

array([4500.0, 4660.0, 4500.0, 4625.0, 0.0, 210, 955350.0,
       Timestamp('1994-11-03 00:00:00'), 0], dtype=object)

In [30]:
def get_class(table_name):
    extend_existing=True
    class GenericTable(Base):
        __tablename__ = table_name

        ID= Column(Integer, primary_key=True)
#         def funcation(self):
#             pass
    return GenericTable

In [100]:


# tstamp=datetime.strptime(df['TIMESTAMP'].iloc[0],'%d-%b-%Y')
# mfrec['id']=mfrec.index
# df['TIMESTAMP']=tstamp
# df
# if 'Unnamed: 11' in df.columns:
#     df=df.drop(['Unnamed: 11'], axis=1)
#     print('Yes it is')
# df
len(dirlist)
file
# 4094, 4095, Zip file not good for file: cm22JUN2011bhav.csv.zip
# Zip file not good for file: cm23JUN2011bhav.csv.zip
df.head(4)

,OPEN,HIGH,LOW,CLOSE,LAST,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES
0,715.0,715.0,715.0,715.00,715.0,100,71500.0,1994-11-03,0
1,4500.0,4660.0,4500.0,4636.75,4625.0,210,955350.0,1994-11-03,0
2,62.0,62.0,62.0,62.00,62.0,100,6200.0,1994-11-03,0
3,425.0,425.0,425.0,425.00,425.0,100,42500.0,1994-11-03,0


In [155]:

dirlist[-10:]

['cm27FEB2018bhav.csv.zip',
 'cm28FEB2018bhav.csv.zip',
 'cm01MAR2018bhav.csv.zip',
 'cm05MAR2018bhav.csv.zip',
 'cm06MAR2018bhav.csv.zip',
 'cm07MAR2018bhav.csv.zip',
 'cm08MAR2018bhav.csv.zip',
 'cm09MAR2018bhav.csv.zip',
 'cm12MAR2018bhav.csv.zip',
 'cm13MAR2018bhav.csv.zip']

In [17]:
# dirlist.sort()
# dirlist

import os
from datetime import datetime
for filename in dirlist:
    info = os.stat("D:/SoftwareWebApps/Python/python_wb/Stocks/Data/"+filename)
    print (datetime.fromtimestamp(info.st_mtime))
    


2018-01-20 09:25:36.384344
2018-01-20 08:55:36.946496
2018-01-20 04:34:09.482402
2018-01-20 04:04:32.574550
2018-01-20 03:35:21.090492
2018-01-20 03:05:04.566205
2018-01-20 08:45:26.279938
2018-01-20 04:53:55.442812
2018-01-20 04:24:04.708156
2018-01-20 03:54:45.698209
2018-01-20 02:55:16.521244
2018-01-20 02:25:47.072958
2018-01-20 09:35:23.683132
2018-01-20 09:05:42.201239
2018-01-20 08:35:31.325376
2018-01-20 03:45:06.457118
2018-01-20 03:15:39.654761
2018-01-20 02:45:23.006925
2018-01-20 02:15:55.431077
2018-01-20 09:30:26.306890
2018-01-20 09:00:45.983607
2018-01-20 08:30:33.201493
2018-01-20 04:38:50.649267
2018-01-20 03:10:16.733533
2018-01-20 02:40:23.153975
2018-01-20 04:41:23.849816
2018-01-20 04:11:47.004485
2018-01-20 03:42:35.811636
2018-01-20 03:12:43.086353
2018-01-20 02:13:28.537547
2018-01-20 09:18:15.387271
2018-01-20 08:48:07.612074
2018-01-20 04:26:37.107669
2018-01-20 03:57:16.776443
2018-01-20 03:28:01.624182
2018-01-20 02:57:43.750047
2018-01-20 09:20:41.549067
2

2018-01-20 03:34:25.236437
2018-01-20 03:04:08.073258
2018-01-20 02:34:38.204353
2018-01-20 09:14:48.566092
2018-01-20 04:52:58.964193
2018-01-20 04:23:06.165542
2018-01-20 03:53:49.718935
2018-01-20 03:24:31.016890
2018-01-20 09:04:46.233249
2018-01-20 08:34:35.342704
2018-01-20 04:42:53.864004
2018-01-20 04:13:17.557670
2018-01-20 02:44:26.266384
2018-01-20 02:14:59.293370
2018-01-20 08:29:37.837634
2018-01-20 04:37:53.831541
2018-01-20 04:08:18.502268
2018-01-20 03:39:07.929349
2018-01-20 02:39:26.260656
2018-01-20 09:31:56.015369
2018-01-20 09:02:15.403914
2018-01-20 08:32:02.980175
2018-01-20 04:10:50.559910
2018-01-20 03:41:39.296815
2018-01-20 03:11:47.130016
2018-01-20 02:41:53.419789
2018-01-20 09:17:19.946730
2018-01-20 04:55:25.168439
2018-01-20 03:27:05.128994
2018-01-20 02:56:47.282565
2018-01-20 02:27:18.281912
2018-01-20 08:49:38.951544
2018-01-20 04:57:51.682262
2018-01-20 04:28:00.749584
2018-01-20 03:58:40.956960
2018-01-20 02:59:14.595248
2018-01-20 02:29:44.930813
2

2018-01-20 02:48:07.069188
2018-01-20 09:10:49.063708
2018-01-20 08:40:41.581394
2018-01-20 04:48:59.432185
2018-01-20 03:20:50.477775
2018-01-20 02:50:33.683290
2018-01-20 02:21:05.954241
2018-01-20 09:23:20.192748
2018-01-20 08:53:20.714011
2018-01-20 04:31:39.838836
2018-01-20 04:02:18.652033
2018-01-20 03:33:06.784129
2018-01-20 03:02:49.467717
2018-01-20 08:43:07.375509
2018-01-20 04:51:40.235928
2018-01-20 04:21:47.765083
2018-01-20 02:53:02.390317
2018-01-20 02:23:31.564376
2018-01-20 09:33:08.486652
2018-01-20 09:03:27.397370
2018-01-20 08:33:16.139384
2018-01-20 03:42:52.620653
2018-01-20 03:13:00.045971
2018-01-20 02:43:07.086852
2018-01-20 02:13:40.306199
2018-01-20 08:28:18.735163
2018-01-20 03:08:00.960055
2018-01-20 09:30:38.261862
2018-01-20 04:39:07.334444
2018-01-20 04:09:31.650722
2018-01-20 03:40:20.573129
2018-01-20 03:10:27.959297
2018-01-20 09:16:01.560885
2018-01-20 08:45:36.960274
2018-01-20 04:24:21.756553
2018-01-20 03:25:45.446982
2018-01-20 02:55:27.714878
2